<a href="https://colab.research.google.com/github/Tuan-LeHoang/hira_kata_recognition/blob/main/Projector_TensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/2023_Project/OCR

/content/drive/MyDrive/2023_Project/OCR


In [ ]:
# !pip install japanize_matplotlib
# !pip install timm

#1. Import libs

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import cv2

import albumentations as A
from albumentations.pytorch import ToTensorV2

import numpy as np # data processing
import matplotlib.pyplot as plt # Data visualization
import japanize_matplotlib
from tqdm import tqdm # Progress bar

import os
import pandas as pd
import torch.nn.functional as F

#2. Load data

In [4]:
list_characters_folders = "./git_lfs_handwritten_hira/hira_better_quality"

In [5]:
data = []
for folder in os.listdir(list_characters_folders):
    if folder == ".DS_Store":
        continue
    character_folder = os.path.join(list_characters_folders, folder)

    for character_image in os.listdir(character_folder):
        if character_image == ".DS_Store":
            continue
        character_image_path = os.path.join(character_folder, character_image)
        data.append([character_image_path, folder])

df = pd.DataFrame(data, columns=['file_name','label'])

In [6]:
sort_df = df.sort_values(by=["label"])


In [7]:
labels = sort_df["label"].unique()
print(labels)

['あ' 'い' 'う' 'え' 'お' 'か' 'き' 'く' 'け' 'こ' 'さ' 'し' 'す' 'せ' 'そ' 'た' 'ち' 'つ'
 'て' 'と' 'な' 'に' 'ぬ' 'ね' 'の' 'は' 'ひ' 'ふ' 'へ' 'ほ' 'ま' 'み' 'む' 'め' 'も' 'や'
 'ゆ' 'よ' 'ら' 'り' 'る' 'れ' 'ろ' 'わ' 'を' 'ん']


##2.1. Split data

In [8]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(sort_df,
                                      test_size = 0.2,
                                      random_state = 42)

##2.2. Convert one-hot

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [10]:
# Convert labels to numeric representation
def convert_to_one_hot_vector_labels(labels):
  label_encoder = LabelEncoder()
  numeric_labels = label_encoder.fit_transform(labels)
  # print("Numeric labels:", numeric_labels)

  # Convert numeric labels to one-hot vectors
  onehot_encoder = OneHotEncoder(sparse=False)
  onehot_labels = onehot_encoder.fit_transform(numeric_labels.reshape(-1, 1))

  return onehot_labels

##2.3. Dataset and Dataloader

In [11]:
from types import SimpleNamespace

cfg = SimpleNamespace(**{})

In [12]:
cfg.root_dir = list_characters_folders
cfg.image_size = 224
cfg.batch_size = 32
cfg.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
cfg.len_embedding_space = len(labels)

In [ ]:
cfg.len_embedding_space

In [13]:
class CustomDataset(Dataset):
    def __init__(self,
                 cfg,
                 df,
                 transform=None,
                  mode = None):
        self.root_dir = cfg.root_dir
        self.df = df
        self.file_names = df['file_name'].values
        self.labels = convert_to_one_hot_vector_labels(df['label'].values)

        if transform and mode != "val":
          self.transform = transform
        elif mode == "train":
          self.transform = A.Compose([A.Resize(cfg.image_size, cfg.image_size),
                             A.Rotate(p=0.6, limit=[-20,20]),
                            #  A.HorizontalFlip(p = 0.6),
                             A.CoarseDropout(max_holes = 1, max_height = 32, max_width = 32, p = 0.3),
                             ToTensorV2()])
        elif mode == "val":
          self.transform = A.Compose([A.Resize(cfg.image_size, cfg.image_size),
                             ToTensorV2()])


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get file_path and label for index
        label = self.labels[idx]
        file_path = self.file_names[idx]
        # file_path = os.path.join(self.root_dir, self.file_names[idx])

        # Read an image with OpenCV
        image = cv2.imread(file_path)

        # Convert the image to RGB color space.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Apply augmentations
        augmented = self.transform(image=image)
        image = augmented['image']

        # Normalize because ToTensorV2() doesn't normalize the image
        image = image/255

        return image, label

In [14]:
train_dataset = CustomDataset(cfg, train_df, mode = "train")
valid_dataset = CustomDataset(cfg, test_df, mode = "val")

train_dataloader = DataLoader(train_dataset,
                          batch_size = cfg.batch_size,
                          shuffle = True)

valid_dataloader = DataLoader(valid_dataset,
                          batch_size = cfg.batch_size,
                          shuffle = False,
                              )

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#3. Model

In [15]:
# import timm

# cfg.n_classes = len(labels)
# cfg.backbone = 'resnet18'

# model = timm.create_model(cfg.backbone,
#                           pretrained = False,
#                           num_classes = cfg.n_classes)

In [16]:
model_path_for_exploring = "./models/1/model.pt"
model_x = torch.load(model_path_for_exploring)

In [17]:
model_x = model_x.to(cfg.device)

In [24]:
model_x.fc = torch.nn.Identity()

#4. Get data

In [18]:
# get some random training images
dataiter = iter(valid_dataloader)
images, labels = next(dataiter)

#5. Set up tensor board

In [35]:
from torch.utils.tensorboard import SummaryWriter
import datetime

In [37]:
log_dir = "logs/tensorboard_learning/projector/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [38]:
writer = SummaryWriter(log_dir = log_dir)

#5. Caculate embedding space

In [32]:
def caculate_embedding_vectors(dataloader, model, cfg):
    # Validation mode
    model.eval()

    embedding_vector_list = []

    # Iterate over data
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        X = batch[0].to(cfg.device)
        y = batch[1].to(cfg.device)

        with torch.no_grad():
            # Forward: Get model outputs
            y_pred = model(X)
            embedding_vector_list.append(y_pred)

    embedding_vector_tensor = torch.cat(embedding_vector_list)

    return embedding_vector_tensor


In [33]:
embedding_vector_tensor = caculate_embedding_vectors(valid_dataloader, model_x, cfg)

100%|██████████| 16/16 [00:01<00:00, 11.78it/s]


In [34]:
embedding_vector_tensor.shape

torch.Size([496, 512])

#6. Get the lables for each images

In [43]:
images_labels = test_df["label"].values

In [45]:
len(images_labels)

496

#7. Create a sprite image

In [48]:
def load_images(dataloader):
    # Validation mode

    list_images = []

    # Iterate over data
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
      X = batch[0]
      list_images.append(X)
    list_images_tensor = torch.cat(list_images)

    return list_images_tensor


In [49]:
list_images_tensor = load_images(valid_dataloader)

100%|██████████| 16/16 [00:01<00:00, 12.06it/s]


In [50]:
list_images_tensor.shape

torch.Size([496, 3, 224, 224])

In [51]:
a = list_images_tensor.unsqueeze(1)

In [52]:
a.shape


torch.Size([496, 1, 3, 224, 224])

#8. Create embedding

In [53]:
writer.add_embedding(embedding_vector_tensor, metadata=images_labels, label_img = list_images_tensor)


In [54]:
writer.flush()

#9. Upload to dev

In [55]:
!tensorboard dev upload --logdir {log_dir} \
  --name "Projector 1" \
  --description "Hiragana" \
  --one_shot

2023-07-14 06:46:06.566694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

logs/tensorboard_learning/projector/20230714-061741

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) y

To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   LKM-SNZ-BWZ



New experiment created. View your TensorBoar

In [59]:
!tensorboard dev upload --logdir logs/tensorboard_learning/projector/20230714-061741 \
  --name "Projector 1" \
  --description "Hiragana" \
  --one_shot

2023-07-14 06:59:25.877074: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/c1BNQIWpRWeaM3bwVOKmSQ/

[2023-07-14T06:59:28] Started scanning logdir.
[2023-07-14T06:59:28] Done scanning logdir.
TensorBoard was run in `one_shot` mode, but did not find any uploadable data in the specified logdir: logs/tensorboard_learning/projector/20230714-061741
An empty experiment was created. To delete the empty experiment you can execute the following

    tensorboard dev delete --experiment_id=c1BNQIWpRWeaM3bwVOKmSQ


Done.


In [ ]:
%load_ext tensorboard
!tensorboard --logdir logs/tensorboard_learning/projector/20230714-061741
# Control TensorBoard display. If no port is provided,
# the most recently launched TensorBoard is used
from tensorboard import notebook
notebook.list() # V

notebook.display(port=6006, height=1000)

In [64]:
!pip install setup_google_colab

ERROR: Could not find a version that satisfies the requirement setup_google_colab (from versions: none)
ERROR: No matching distribution found for setup_google_colab
